# Imports, setting up device and seeds

In [1]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
 
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# torch.cuda.set_device(1)
# device = torch.device("cuda")
# device = torch.device('mps')
device = torch.device('cpu')

In [3]:
# torch.manual_seed(42)


import os
import random

def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_all_seeds(42)

# Constants

In [4]:
GOOGLE_COLAB_VERSION = False
VALIDATION_PERCENTAGE = 0.10
TRAIN_PATH = "./train"
TEST_PATH = "./test_all"

# Get data

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainFolder = torchvision.datasets.ImageFolder(root=TRAIN_PATH,
                                               transform=transform)

# trainloader = torch.utils.data.DataLoader(trainFolder, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root=TEST_PATH, train=False,
#                                        download=False, transform=transform)

# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)

In [6]:
# split into train and validation
train_size = int((1 - VALIDATION_PERCENTAGE) * len(trainFolder))
val_size = len(trainFolder) - train_size
train_subset, val_subset = torch.utils.data.random_split(trainFolder, [train_size, val_size])

In [7]:
len(train_subset.indices)

79209

In [8]:
len(val_subset.indices)

8802

In [9]:
# create dataloaders
train_dataloader = torch.utils.data.DataLoader(train_subset, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_subset, batch_size=256) # TODO: change to max, który się szybko liczy

In [10]:
test_batch = next(iter(train_dataloader))

In [11]:
test_batch[0].shape

torch.Size([32, 3, 64, 64])

In [12]:
classes = os.listdir(TRAIN_PATH)

In [13]:
# class Net(nn.Mobdule):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
#         self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv2 = nn.Conv2d(64, 128, 2)
#         self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv3 = nn.Conv2d(128, 256, 2)
#         self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv4 = nn.Conv2d(256, 512, 2)
#         self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

#         self.conv5 = nn.Conv2d(64, 128, 2)
#         self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv6 = nn.Conv2d(128, 128, 2)
#         self.pool6 = nn.MaxPool2d(kernel_size=2, stride=1)
#         self.conv7 = nn.Conv2d(128, 128, 2)
#         self.pool7 = nn.MaxPool2d(kernel_size=2, stride=1)
#         self.conv8 = nn.Conv2d(128, 128, 2)
#         self.pool8 = nn.MaxPool2d(kernel_size=2, stride=1)

#         self.fc1 = nn.Linear(4608, 1024)
#         self.fc2 = nn.Linear(1024, 50)

#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv1(x)))
#         x = self.pool2(F.relu(self.conv2(x)))
#         x = self.pool3(F.relu(self.conv3(x)))
#         x = self.pool4(F.relu(self.conv4(x)))

#         # x = self.pool5(F.relu(self.conv5(x)))
#         # x = self.pool6(F.relu(self.conv6(x)))
#         # x = self.pool7(F.relu(self.conv7(x)))
#         # x = self.pool8(F.relu(self.conv8(x)))

#         # print(x.shape, x.shape[1] * x.shape[2] * x.shape[3])
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

In [14]:
# class LongNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         ## Warstwa konwolucyjna
#         self.n_convs =  20
#         n_channels = 10
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=n_channels, kernel_size=5, stride=1, padding=0)
#         self.conv2 = nn.Conv2d(in_channels=n_channels, out_channels=n_channels, kernel_size=5, stride=1, padding=0)
#         self.convs = [nn.Conv2d(in_channels=n_channels*2, out_channels=n_channels, kernel_size=5, stride=1, padding=2) for _ in range(self.n_convs)]
#         self.bns = [nn.BatchNorm2d(n_channels) for _ in range(self.n_convs)]
     
#         ## Warstwa max pooling 
#         self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

#         self.fc1 = nn.Linear(n_channels * 28 * 28, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 50)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x_2 = F.relu(self.conv2(x))
#         x = x_2
#         for i in range(self.n_convs):    # tutaj w pętli jest chyba skip connection, skipujemy wyjścia drugiej warstwy do kazdej kolejnej
#             x = torch.cat([x,x_2],dim=1)
#             x = F.relu(self.convs[i](x))
#             x = self.bns[i](x)
#         x = self.pool1(x)
# #         print(x.size())
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


# net = LongNet().to(device)
# for conv in net.convs:
#     conv.to(device)
# for bn in net.bns:
#     bn.to(device)
# net

In [15]:
# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         ## Warstwa konwolucyjna
#         self.n_convs =  8
#         n_channels = 10
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=n_channels, kernel_size=5, stride=1, padding=0)
#         self.conv2 = nn.Conv2d(in_channels=n_channels, out_channels=n_channels, kernel_size=5, stride=1, padding=0)

#         self.conv3 = nn.Conv2d(in_channels=n_channels*2, out_channels=n_channels, kernel_size=5, stride=1, padding=2)
#         self.bn3 = nn.BatchNorm2d(n_channels)
#         self.conv4 = nn.Conv2d(in_channels=n_channels*2, out_channels=n_channels, kernel_size=5, stride=1, padding=2)
#         self.bn4 = nn.BatchNorm2d(n_channels)
#         self.conv5 = nn.Conv2d(in_channels=n_channels*2, out_channels=n_channels, kernel_size=5, stride=1, padding=2)
#         self.bn5 = nn.BatchNorm2d(n_channels)
#         self.conv6 = nn.Conv2d(in_channels=n_channels*2, out_channels=n_channels, kernel_size=5, stride=1, padding=2)
#         self.bn6 = nn.BatchNorm2d(n_channels)
#         self.conv7 = nn.Conv2d(in_channels=n_channels*2, out_channels=n_channels, kernel_size=5, stride=1, padding=2)
#         self.bn7 = nn.BatchNorm2d(n_channels)
#         self.conv8 = nn.Conv2d(in_channels=n_channels*2, out_channels=n_channels, kernel_size=5, stride=1, padding=2)
#         self.bn8 = nn.BatchNorm2d(n_channels)
#         self.conv9 = nn.Conv2d(in_channels=n_channels*2, out_channels=n_channels, kernel_size=5, stride=1, padding=2)
#         self.bn9 = nn.BatchNorm2d(n_channels)
#         self.conv10 = nn.Conv2d(in_channels=n_channels*2, out_channels=n_channels, kernel_size=5, stride=1, padding=2)
#         self.bn10 = nn.BatchNorm2d(n_channels)

#         self.convs = [self.conv3, self.conv4, self.conv5, self.conv6, self.conv7, self.conv8, self.conv9, self.conv10]
#         self.bns = [self.bn3, self.bn4, self.bn5, self.bn6, self.bn7, self.bn8, self.bn9, self.bn10]
#         # self.convs = [nn.Conv2d(in_channels=n_channels*2, out_channels=n_channels, kernel_size=5, stride=1, padding=2) for _ in range(self.n_convs)]
#         # self.bns = [nn.BatchNorm2d(n_channels) for _ in range(self.n_convs)]

#         ## Warstwa max pooling 
#         self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

#         self.fc1 = nn.Linear(n_channels * 28 * 28, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 50)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x_2 = F.relu(self.conv2(x))
#         x = x_2
#         for i in range(self.n_convs):    # tutaj w pętli jest chyba skip connection, skipujemy wyjścia drugiej warstwy do kazdej kolejnej
#             x = torch.cat([x,x_2],dim=1)
#             x = F.relu(self.convs[i](x))
#             x = self.bns[i](x)
#         x = self.pool1(x)
# #         print(x.size())
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


In [72]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels=64, out_channels=128):
        super().__init__()
        ## Warstwa konwolucyjna
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.conv4 = nn.Conv2d(in_channels=in_channels*2, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        x_1 = self.bn1(F.relu(self.conv1(x)))
        # x = x_1

        x_2 = self.bn2(F.relu(self.conv2(x_1)))
        # x = x_2

        x_3 = self.bn3(F.relu(self.conv3(x_2)))
        x_3 = torch.cat([x_3, x],dim=1)
        # x = torch.cat([x_3, x],dim=1)

        x_4 = self.bn3(F.relu(self.conv4(x_3)))

        x_5 = torch.cat([x_4, x_2],dim=1)
        # x = x_2
        # for i in range(self.n_convs):    # tutaj w pętli jest chyba skip connection, skipujemy wyjścia drugiej warstwy do kazdej kolejnej
        #     x = torch.cat([x,x_2],dim=1)
        #     x = F.relu(self.convs[i](x))
        #     x = self.bns[i](x)
        # x = self.pool1(x)
#         print(x.size())
        # x = torch.flatten(x, 1) # flatten all dimensions except batch
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        return x_5


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        ## Warstwa konwolucyjna
        n_channels = 64
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=n_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=n_channels, out_channels=n_channels, kernel_size=3, stride=1, padding=1)

        self.conv_block1 = ConvBlock(in_channels=n_channels, out_channels=n_channels)

        # self.convs = [self.conv3, self.conv4, self.conv5, self.conv6, self.conv7, self.conv8, self.conv9, self.conv10]
        # self.bns = [self.bn3, self.bn4, self.bn5, self.bn6, self.bn7, self.bn8, self.bn9, self.bn10]
        # self.convs = [nn.Conv2d(in_channels=n_channels*2, out_channels=n_channels, kernel_size=5, stride=1, padding=2) for _ in range(self.n_convs)]
        # self.bns = [nn.BatchNorm2d(n_channels) for _ in range(self.n_convs)]

        ## Warstwa max pooling 
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(n_channels * 28 * 28, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 50)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv_block1(x))
        x = self.pool1(x)
#         print(x.size())
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [73]:
import torch

n_channels_test = 64
input_test = torch.rand(32, 3, 32, 32)

conv1_test = torch.nn.Conv2d(in_channels=3, out_channels=n_channels_test, kernel_size=3, stride=1, padding=1)
x_test = conv1_test(input_test)
print(x_test.shape)

conv2_test = nn.Conv2d(in_channels=n_channels_test, out_channels=n_channels_test, kernel_size=3, stride=1, padding=1)
x_test = conv2_test(x_test)
print(x_test.shape)

conv3_test = ConvBlock(in_channels=n_channels_test)
x_test = conv3_test(x_test)
print(x_test.shape)

torch.Size([32, 64, 32, 32])
torch.Size([32, 64, 32, 32])


RuntimeError: Given groups=1, weight of size [128, 64, 3, 3], expected input[32, 128, 32, 32] to have 64 channels, but got 128 channels instead

In [60]:
import torchvision.models as models

# net = Net().to(device)
# net = models.resnet18().to(device)
# net = models.densenet161().to(device)
net = Net().to(device)
net

Net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_block1): ConvBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_featu

In [61]:
print(test_batch)
print(type(test_batch))
print(len(test_batch))

[tensor([[[[-0.2078, -0.2392, -0.3020,  ..., -0.1765, -0.1373, -0.1686],
          [-0.1765, -0.2000, -0.2706,  ..., -0.2706, -0.1843, -0.1765],
          [-0.1373, -0.1608, -0.2235,  ..., -0.2549, -0.1608, -0.1451],
          ...,
          [-0.3961, -0.3725, -0.3647,  ..., -0.4588, -0.4510, -0.4824],
          [-0.4431, -0.4039, -0.3961,  ..., -0.5137, -0.5451, -0.4980],
          [-0.5059, -0.4667, -0.4510,  ..., -0.5373, -0.6157, -0.4824]],

         [[-0.4118, -0.4196, -0.4353,  ...,  0.0118,  0.0431,  0.0118],
          [-0.3882, -0.4039, -0.4118,  ..., -0.0824, -0.0039,  0.0039],
          [-0.3804, -0.3882, -0.3961,  ..., -0.0667,  0.0275,  0.0353],
          ...,
          [-0.1608, -0.1373, -0.1294,  ..., -0.4196, -0.4196, -0.4588],
          [-0.2235, -0.1843, -0.1608,  ..., -0.4510, -0.5137, -0.4667],
          [-0.2863, -0.2471, -0.2314,  ..., -0.4745, -0.5843, -0.4510]],

         [[-0.5451, -0.5608, -0.5922,  ..., -0.2549, -0.1843, -0.2078],
          [-0.5216, -0.5373, 

In [62]:
from torchviz import make_dot
from datetime import datetime

# uncomment this cell to visualize a net (do it on the cpu - change device from 'mps' to 'cpu')
yhat = net(test_batch[0])
current_datetime_formatted = f'{datetime.now()}'.replace('-', '_').replace(' ', '_').replace(':', '_')[:-4]
make_dot(yhat, params=dict(list(net.named_parameters()))).render(f"net_visualizations/net_visualization{current_datetime_formatted}", format="png")

RuntimeError: Given groups=1, weight of size [64, 64, 3, 3], expected input[32, 128, 64, 64] to have 64 channels, but got 128 channels instead

In [ ]:
def number_of_parameters(net=models.resnet18()):
    params_sum = 0
    for params in net.parameters():
        params_sum+=params.view(-1).size(0)
    return params_sum

In [ ]:
# number of parameters in Net - compare to resnet18
print(f'number of parameters in created net = {number_of_parameters(net=net)}')
print(f'number of parameters in resnet18    = {number_of_parameters()}')

number of parameters in created net = 998844
number of parameters in resnet18    = 11689512


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [ ]:
def get_accuracy(val_dataloader=val_dataloader):
    # copy and create new get_accuracy() func - per class
    val_correct_count = 0
    val_data_len = 0

    for i, data in enumerate(val_dataloader, 0):
        net.eval()
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # forward + backward + optimize
        outputs = net(inputs)

        outputs_for_val_correct_count = outputs.max(1, keepdim=True)[1].squeeze()
        val_correct_count += (outputs_for_val_correct_count == labels).sum()
        val_data_len += len(labels)

    new_val_acc = val_correct_count / val_data_len
    return new_val_acc

In [ ]:
%%time

epochs, losses, train_acc, val_acc = [], [], [], []

NUM_EPOCHS = 5
for epoch in range(NUM_EPOCHS): 
    train_correct_count = 0
    train_data_len = 0
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # print(i)
        net.train()

        print(data)
        print(type(data))
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        outputs_for_train_correct_count = outputs.max(1, keepdim=True)[1].squeeze()
        train_correct_count += (outputs_for_train_correct_count == labels).sum()
        train_data_len += len(labels)

        running_loss_single_iter = loss.item()
        running_loss += running_loss_single_iter

    new_train_acc = train_correct_count / train_data_len
    train_acc.append(new_train_acc)

    print(f'epoch {epoch} train accuracy = {new_train_acc}, val_accuracy = {get_accuracy()}')

    print(f'[{epoch+1}/{NUM_EPOCHS}] loss: {running_loss/1000}')
    running_loss = 0.0

print('Finished Training')

[tensor([[[[-0.4667, -0.4588, -0.4510,  ..., -0.3882, -0.4275, -0.4588],
          [-0.4745, -0.4667, -0.4431,  ..., -0.3961, -0.4275, -0.4510],
          [-0.4745, -0.4588, -0.4275,  ..., -0.3725, -0.3961, -0.4196],
          ...,
          [-0.4196, -0.3098, -0.2627,  ..., -0.1922, -0.2157, -0.2000],
          [-0.3569, -0.2863, -0.2784,  ..., -0.2078, -0.2549, -0.2706],
          [-0.3490, -0.3333, -0.3725,  ..., -0.1608, -0.2157, -0.2392]],

         [[-0.4902, -0.4824, -0.4745,  ..., -0.4118, -0.4510, -0.4824],
          [-0.4980, -0.4902, -0.4667,  ..., -0.4196, -0.4510, -0.4745],
          [-0.4980, -0.4824, -0.4510,  ..., -0.3961, -0.4196, -0.4431],
          ...,
          [-0.4275, -0.3176, -0.2706,  ..., -0.2000, -0.2235, -0.2078],
          [-0.3647, -0.2941, -0.2863,  ..., -0.2157, -0.2627, -0.2784],
          [-0.3569, -0.3412, -0.3804,  ..., -0.1686, -0.2235, -0.2471]],

         [[-0.5294, -0.5216, -0.5137,  ..., -0.4510, -0.4902, -0.5216],
          [-0.5373, -0.5294, 

In [ ]:
print(f'validation accuracy = {get_accuracy()}')

validation accuracy = 0.01999545469880104
